In [ ]:
import openai, json

client = openai.OpenAI()
message = []

In [ ]:
def get_weather(city):
    return '33 degrees celsius. '

FUNCTION_MAP = {
    "get_weather": get_weather
}

In [ ]:
TOOLS = [
    {
        'type': 'function',
        'function': {
            'name': 'get_weather',
            'description': 'Get the current weather in a given city',
            'parameters': {
                'type': 'object',
                'properties': {
                    'city': {
                        'type': 'string',
                        'description': 'The name of the city to get the weather of.'
                    }
                },
                'required': ['city']
            }
        }
    }
]

'''
[TOOLS] 를 추가하여 요청으로 보냄

1. 사용자가 요청을 보낼 때 가지고 있는 함수(function)에 설명(description)을 같이 보냄

2. AI는 사용자의 요청과 함수를 함께 보고, 사용자의 함수에서 실행하고 싶은 함수가 있으면 message.tool_calls에 담아서 보냄.

3. message.tool_calls에 내용이 있다면 messages에 추가하고(이력을 남겨야 하므로), 다시 AI에게 요청.
'''

In [ ]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    
    if message.tool_calls:  # 1. AI한테 받은 메시지 안에 tool_calls 가 있는지 확인하고,
        message.append({ # 2. 그 요청을 메모리에 저장
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    'id': tool_call.id,
                    'type': 'function',
                    'function': {
                        'name': tool_call.function.name,
                        'arguments': tool_call.function.arguments # <-- 여기서 arguments 는 사실 그냥 문자열.. {"city": "Seoul"} 형식
                    }
                }
                for tool_call in message.tool_calls
            ],
        })

        for tool_call in message.tool_calls:  # 3. AI가 실행하려는 각 tool_call(함수 호출) 마다 함수 이름과 arguments(인자)를 뽑아서
            function_name = tool_call.function.name
            function_args = tool_call.function.arguments

            print(f"Calling function: {function_name} with args: {function_args}")

            try:
                arguments = json.loads(arguments) # <-- 모델에서 '{"city": "Seoul"}' 이런 식으로 문자열을 받아서 이를 python 딕셔너리로 바꾸고, JSON으로 로드
            except json.JSONDecodeError:
                arguments = {}  # 4. 그 문자열을 Python 딕셔너리로 바꾸고 

            function_to_run = FUNCTION_MAP.get(function_name)  # 5. AI가 요청한 함수 이름으로 function_map에서 함수를 가져옴

            # print(f"Ran {function_name} with args {arguments} for a result of {result}")

            result = function_to_run(**arguments) # <-- ** ? 그 딕셔너리를 키워드 인자로 바꿔주는 역할.. {"city": "Seoul"} -> city="Seoul" => get_weather(city="Seoul")
            # 6. 그 함수를 arguments 키워드 인자로 넘겨서 실행

            message.append({  # 7. message 메모리에 또 하나의 메시지 추가
                "role": "tool",
                "tool_call_id": tool_call.id,
                'name': function_name,
                "content": result,
            })

        call_ai()  # 8. 일련의 흐름 후에 여기서 ai를 한번 더 call. 그 후 def call_ai()를 실행.
    else:
        message.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")  # 9. 이 두 개의 else 코드가 실행되면 sequence가 종료되고

def call_ai():
    response = client.chat.completions.create(
        model='gpt-5-nano',
        messages = message,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)
    

In [ ]:
while True: # 10. 다시 처음부터
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        message.append({
            "role": "user",
            "content": message
        })
        print(f"User: {message}")
        call_ai()